# Exploring Ebay Car Sales Data

In this project we'll be exploring some car sale [data](https://data.world/data-society/used-cars-data) from the German eBay website. The dataset we'll use in this project is a sample from the original dataset that contains 50,000 data points and has been dirtied to simulate a scraped dataset.

Our goal for this project is to clean all the bad data, and then do some analysis on some of the columns.

In [1]:
# As this dataset contains a data frame structure, we'll need to import pandas and NumPy
import numpy as np
import pandas as pd

# This dataframe is encoded with Latin-1, so we'll need to specify this
autos = pd.read_csv(r'C:\Users\Green Miracle\csv files\autos.csv', encoding='latin1')

Let's first explore our data and get an idea of what we'll be working with. We'll take a look at the first first rows, as we'll as some useful info on each column.

In [2]:
# The display() function will let us visualize the dataframe
display(autos.head(5))

# Using the info method, we can view useful column information
autos.info()

,dateCrawled,name,seller,offerType,price,abtest,vehicleType,yearOfRegistration,gearbox,powerPS,model,odometer,monthOfRegistration,fuelType,brand,notRepairedDamage,dateCreated,nrOfPictures,postalCode,lastSeen
0,2016-03-26 17:47:46,Peugeot_807_160_NAVTECH_ON_BOARD,privat,Angebot,"$5,000",control,bus,2004,manuell,158,andere,"150,000km",3,lpg,peugeot,nein,2016-03-26 00:00:00,0,79588,2016-04-06 06:45:54
1,2016-04-04 13:38:56,BMW_740i_4_4_Liter_HAMANN_UMBAU_Mega_Optik,privat,Angebot,"$8,500",control,limousine,1997,automatik,286,7er,"150,000km",6,benzin,bmw,nein,2016-04-04 00:00:00,0,71034,2016-04-06 14:45:08
2,2016-03-26 18:57:24,Volkswagen_Golf_1.6_United,privat,Angebot,"$8,990",test,limousine,2009,manuell,102,golf,"70,000km",7,benzin,volkswagen,nein,2016-03-26 00:00:00,0,35394,2016-04-06 20:15:37
3,2016-03-12 16:58:10,Smart_smart_fortwo_coupe_softouch/F1/Klima/Pan...,privat,Angebot,"$4,350",control,kleinwagen,2007,automatik,71,fortwo,"70,000km",6,benzin,smart,nein,2016-03-12 00:00:00,0,33729,2016-03-15 03:16:28
4,2016-04-01 14:38:50,Ford_Focus_1_6_Benzin_TÜV_neu_ist_sehr_gepfleg...,privat,Angebot,"$1,350",test,kombi,2003,manuell,0,focus,"150,000km",7,benzin,ford,nein,2016-04-01 00:00:00,0,39218,2016-04-01 14:38:50


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 50000 entries, 0 to 49999
Data columns (total 20 columns):
 #   Column               Non-Null Count  Dtype 
---  ------               --------------  ----- 
 0   dateCrawled          50000 non-null  object
 1   name                 50000 non-null  object
 2   seller               50000 non-null  object
 3   offerType            50000 non-null  object
 4   price                50000 non-null  object
 5   abtest               50000 non-null  object
 6   vehicleType          44905 non-null  object
 7   yearOfRegistration   50000 non-null  int64 
 8   gearbox              47320 non-null  object
 9   powerPS              50000 non-null  int64 
 10  model                47242 non-null  object
 11  odometer             50000 non-null  object
 12  monthOfRegistration  50000 non-null  int64 
 13  fuelType             45518 non-null  object
 14  brand                50000 non-null  object
 15  notRepairedDamage    40171 non-null  object
 16  date

From this information, I can gather a few key points.
* Written in German
* Columns contain CamelCase
* The odometer and price column seem to contain mostly numeric data, but are an object dtype
* 5 columns contain null values

## Data Cleaning: Column Names

To begin, let's first change the CamelCase columns into the preferred snake_case, and make some of the columns more descriptive, which will make our data much easier to handle.

In [3]:
# Let's create a new list to hold all of our updated names
new_cols = []

# To make converting CamelCase to snake_case easier, I'll make a simple function
def camel_to_snake(label):
    fixed = ""
    # The flag is so that columns with uppercase letters side-by-side don't get an unnecessary underscore
    flag = True
    
    for l in label:
        # If the letter is uppercase, and the flag is True, then add an underscore
        # before the letter and make the flag False
        if l.isupper() and flag == True:
            fixed += "_"
            fixed += l
            flag = False
        # Else if the letter is lowercase, add the letter and make the flag True
        else:
            fixed += l
            flag = True
            
    return fixed

# To populate our new list, we'll loop through every column label
for l in autos.columns:
    # We'll manually change a few of the labels to make them easier to read
    l = l.replace('yearOfRegistration', 'registration_year')
    l = l.replace('monthOfRegistration', 'registration_month')
    l = l.replace('notRepairedDamage', 'unrepaired_damage')
    l = l.replace('dateCreated', 'ad_created')
    # Then use our new function and make everything lowercase
    l = camel_to_snake(l)
    l = l.lower()
    # And finally, add each label to our list
    new_cols.append(l)
    
# And now use the new labels in our list in place of each column's current label
autos.columns = new_cols

# Let's view our results
print(autos.columns)

Index(['date_crawled', 'name', 'seller', 'offer_type', 'price', 'abtest',
       'vehicle_type', 'registration_year', 'gearbox', 'power_ps', 'model',
       'odometer', 'registration_month', 'fuel_type', 'brand',
       'unrepaired_damage', 'ad_created', 'nr_of_pictures', 'postal_code',
       'last_seen'],
      dtype='object')


Now the columns are uniform and much easier to read. With this, we'll be able to refer to columns in our code much easier without worrying about which letters are uppercase and which are not.

## Column Overview

Let's take a closer look at the information in all the columns. This will give us an idea of what columns need to be looked at, removed, and changed.

In [4]:
# We'll use the describe method to view some statistics for every column
autos.describe(include='all')

,date_crawled,name,seller,offer_type,price,abtest,vehicle_type,registration_year,gearbox,power_ps,model,odometer,registration_month,fuel_type,brand,unrepaired_damage,ad_created,nr_of_pictures,postal_code,last_seen
count,50000,50000,50000,50000,50000,50000,44905,50000.000000,47320,50000.000000,47242,50000,50000.000000,45518,50000,40171,50000,50000.0,50000.000000,50000
unique,48213,38754,2,2,2357,2,8,NaN,2,NaN,245,13,NaN,7,40,2,76,NaN,NaN,39481
top,2016-03-23 19:38:20,Ford_Fiesta,privat,Angebot,$0,test,limousine,NaN,manuell,NaN,golf,"150,000km",NaN,benzin,volkswagen,nein,2016-04-03 00:00:00,NaN,NaN,2016-04-07 06:17:27
freq,3,78,49999,49999,1421,25756,12859,NaN,36993,NaN,4024,32424,NaN,30107,10687,35232,1946,NaN,NaN,8
mean,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2005.073280,NaN,116.355920,NaN,NaN,5.723360,NaN,NaN,NaN,NaN,0.0,50813.627300,NaN
std,NaN,NaN,NaN,NaN,NaN,NaN,NaN,105.712813,NaN,209.216627,NaN,NaN,3.711984,NaN,NaN,NaN,NaN,0.0,25779.747957,NaN
min,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1000.000000,NaN,0.000000,NaN,NaN,0.000000,NaN,NaN,NaN,NaN,0.0,1067.000000,NaN
25%,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1999.000000,NaN,70.000000,NaN,NaN,3.000000,NaN,NaN,NaN,NaN,0.0,30451.000000,NaN
50%,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2003.000000,NaN,105.000000,NaN,NaN,6.000000,NaN,NaN,NaN,NaN,0.0,49577.000000,NaN
75%,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2008.000000,NaN,150.000000,NaN,NaN,9.000000,NaN,NaN,NaN,NaN,0.0,71540.000000,NaN


I notice that:

* The seller and offer_type columns contain nearly all of the same value. These columns contains no useful information, and can be removed.
* The nr_of_pictures column seem to only contain values of 0, so we'll wanna look into that.
* odometer and price contain numeric data as an object dtype. These columns will be easier to analyze as a numeric dtype.

## Data Cleaning: Columns

To begin, let's take a look at the nr_of_pictures column. This column may be a candidate for removal if it contains no useful information.

In [5]:
# We'll count the number of times 0 is in this column
autos['nr_of_pictures'].value_counts()

0    50000
Name: nr_of_pictures, dtype: int64

Every row contains 0. This is useless to our analysis, so we'll remove this column along with seller and offer_type.

In [6]:
autos = autos.drop(['seller', 'offer_type', 'nr_of_pictures'], axis=1)
print(autos.columns)

Index(['date_crawled', 'name', 'price', 'abtest', 'vehicle_type',
       'registration_year', 'gearbox', 'power_ps', 'model', 'odometer',
       'registration_month', 'fuel_type', 'brand', 'unrepaired_damage',
       'ad_created', 'postal_code', 'last_seen'],
      dtype='object')


Next let's convert the price and odometer columns to a numeric datatype. To do this we'll first have to remove the string characters.

In [7]:
# We'll use the vectorized string replace method to remove the non-numeric values from both columns
autos["price"] = autos["price"].str.replace("$", "")
autos["price"] = autos["price"].str.replace(",", "").astype("int64")
print(autos["price"].head())

autos["odometer"] = autos["odometer"].str.replace("km", "")
autos["odometer"] = autos["odometer"].str.replace(",", "").astype("int64")
print(autos["odometer"].head())

0    5000
1    8500
2    8990
3    4350
4    1350
Name: price, dtype: int64
0    150000
1    150000
2     70000
3     70000
4    150000
Name: odometer, dtype: int64


The odometer column should now be renamed to reflect the units used to measure the column, as this information is no longer obvious.

In [8]:
autos.rename({"odometer":"odometer_km"}, axis=1, inplace=True)
print(autos.columns)

Index(['date_crawled', 'name', 'price', 'abtest', 'vehicle_type',
       'registration_year', 'gearbox', 'power_ps', 'model', 'odometer_km',
       'registration_month', 'fuel_type', 'brand', 'unrepaired_damage',
       'ad_created', 'postal_code', 'last_seen'],
      dtype='object')


## Data Cleaning: English

To make our data more easily understandable for English readers, I'll convert all the German text to English for each column.

### vehicle_type

In [9]:
autos.loc[autos["vehicle_type"] == 'andere', 'vehicle_type'] = "other"
autos.loc[autos["vehicle_type"] == 'kleinwagen', 'vehicle_type'] = "subcompact"
autos.loc[autos["vehicle_type"] == 'cabrio', 'vehicle_type'] = "convertible"
autos.loc[autos["vehicle_type"] == 'kombi', 'vehicle_type'] = "combi"

autos['vehicle_type'].value_counts()

limousine      12859
subcompact     10822
combi           9127
bus             4093
convertible     3061
coupe           2537
suv             1986
other            420
Name: vehicle_type, dtype: int64

### gearbox

In [10]:
autos.loc[autos["gearbox"] == 'manuell', 'gearbox'] = "manual"
autos.loc[autos["gearbox"] == 'automatik', 'gearbox'] = "automatic"

autos['gearbox'].value_counts()

manual       36993
automatic    10327
Name: gearbox, dtype: int64

### model

In [11]:
autos.loc[autos["model"] == 'andere', 'model'] = "other"

### fuel_type

In [12]:
autos.loc[autos["fuel_type"] == 'benzin', 'fuel_type'] = "petrol"
autos.loc[autos["fuel_type"] == 'andere', 'fuel_type'] = "other"
autos.loc[autos["fuel_type"] == 'elektro', 'fuel_type'] = "electric"

autos['fuel_type'].value_counts()

petrol      30107
diesel      14567
lpg           691
cng            75
hybrid         37
other          22
electric       19
Name: fuel_type, dtype: int64

### unrepaired_damage

In [13]:
autos.loc[autos["unrepaired_damage"] == 'nein', 'unrepaired_damage'] = "no"
autos.loc[autos["unrepaired_damage"] == 'ja', 'unrepaired_damage'] = "yes"

autos['unrepaired_damage'].value_counts()

no     35232
yes     4939
Name: unrepaired_damage, dtype: int64

## Data Cleaning: Outliers

### Price

Let's take a look at the price column for any possible outliers. In our overview above, we noticed that there were 1,400 rows with a value of 0. This seems strange for a car sale, but may just be a common price to start auctions on.

In [14]:
# To begin, we'll count the number of unique values
print("Number of unique values:" "\n", 
      autos["price"].unique().shape
      ,"\n", sep="")

# View the series statistics
print("Price statistics:" "\n", 
      autos["price"].describe(percentiles=[.05, .25, .50, .75, .99])
     ,"\n", sep="")

# And view the outliers
print("Five Lowest Prices:" "\n",
      autos["price"].value_counts().sort_index(ascending=True).head()
      ,"\n", sep="")
print("Five Highest Prices:" "\n",
      autos["price"].value_counts().sort_index(ascending=False).head()
      ,"\n", sep="")

Number of unique values:
(2357,)

Price statistics:
count    5.000000e+04
mean     9.840044e+03
std      4.811044e+05
min      0.000000e+00
5%       2.000000e+02
25%      1.100000e+03
50%      2.950000e+03
75%      7.200000e+03
99%      3.590000e+04
max      1.000000e+08
Name: price, dtype: float64

Five Lowest Prices:
0    1421
1     156
2       3
3       1
5       2
Name: price, dtype: int64

Five Highest Prices:
99999999    1
27322222    1
12345678    3
11111111    2
10000000    1
Name: price, dtype: int64



With our results, we can see that the highest prices cost over 10 million dollars. This seems rather unrealistic for an eBay car sale, and are unlikely to be actual sales.

Since 99% of the listing prices are under 36,000, we'll only include listing that are under 50,000 to remove the unlikely outliers. We'll keep the listings of 0$, as they may just be a popular auction starting price.

In [15]:
# We'll remove the rows with a price over 50000 by using boolean indexing
autos = autos[autos['price'] <= 50000]

print(autos["price"].describe())

count    49800.000000
mean      5413.582892
std       6684.176349
min          0.000000
25%       1100.000000
50%       2900.000000
75%       7000.000000
max      50000.000000
Name: price, dtype: float64


We removed 200 outliers that were over 50000. Now our data will be much more accurate. With our new data, we can see that the average cost is about 5400.

### Odometer

Let's also take a quick look at the odometer_km column, which only contains 13 unique values. 

In [16]:
# Since there are only 13 unique values, let's just view the number of listings for each value
print(autos["odometer_km"].value_counts().sort_index(ascending=True))

5000        944
10000       251
20000       755
30000       765
40000       800
50000      1014
60000      1148
70000      1224
80000      1434
90000      1750
100000     2158
125000     5158
150000    32399
Name: odometer_km, dtype: int64


It appears that the listings are rounded into these unique values, and the exact milage is not posted. With this, we can see that more than half of the listings contain vehicles with over 150,000km. 

## Analysis: Dates

This dataset contains five columns that contain dates.
* date_crawled
* last_seen
* ad_created
* registration_month
* registration_year

Three of these columns are represented as strings, which contain the date and time. This values can easily be distributed by only including the date portion and sorting the values by the index. This will allow us to get an understanding of the range of dates in each column.

We'll first take a look at the date_crawled column, which includes the date the ad was collected by the crawler.

In [17]:
# We'll view the distribution of every unique date as a percentage by including the `normalize=True` parameter
print(autos['date_crawled'].str[:10].value_counts(normalize=True, dropna=False))

2016-04-03    0.038675
2016-03-20    0.037892
2016-03-21    0.037450
2016-03-12    0.036827
2016-03-14    0.036687
2016-04-04    0.036486
2016-03-07    0.035964
2016-04-02    0.035361
2016-03-19    0.034920
2016-03-28    0.034839
2016-03-29    0.034177
2016-03-15    0.033996
2016-04-01    0.033775
2016-03-30    0.033675
2016-03-08    0.033273
2016-03-09    0.033233
2016-03-22    0.032871
2016-03-26    0.032510
2016-03-11    0.032450
2016-03-23    0.032329
2016-03-10    0.032229
2016-03-31    0.031888
2016-03-25    0.031727
2016-03-17    0.031506
2016-03-27    0.030984
2016-03-16    0.029498
2016-03-24    0.029116
2016-03-05    0.025402
2016-03-13    0.015602
2016-03-06    0.013976
2016-03-18    0.013052
2016-04-05    0.013012
2016-04-06    0.003193
2016-04-07    0.001426
Name: date_crawled, dtype: float64


It appears that the crawler collected data every day for a little over a month. Most days seem to have a similar amount of data collected, aside from the last two days in particular. These are the last two days the crawler ran, and it may have just had less new data available to collect.

Let's take a look at the last_seen column now, which is when the ad was last seen by the crawler.

In [18]:
# We'll sort this column by the dates
print(autos['last_seen'].str[:10].value_counts(normalize=True, dropna=False).sort_index(ascending=True))

2016-03-05    0.001084
2016-03-06    0.004438
2016-03-07    0.005341
2016-03-08    0.007610
2016-03-09    0.009839
2016-03-10    0.010803
2016-03-11    0.012570
2016-03-12    0.023835
2016-03-13    0.009016
2016-03-14    0.012831
2016-03-15    0.015904
2016-03-16    0.016426
2016-03-17    0.028032
2016-03-18    0.007410
2016-03-19    0.015783
2016-03-20    0.020683
2016-03-21    0.020783
2016-03-22    0.021627
2016-03-23    0.018594
2016-03-24    0.019558
2016-03-25    0.019237
2016-03-26    0.016968
2016-03-27    0.016024
2016-03-28    0.020884
2016-03-29    0.022410
2016-03-30    0.024880
2016-03-31    0.023835
2016-04-01    0.023173
2016-04-02    0.024940
2016-04-03    0.025382
2016-04-04    0.024719
2016-04-05    0.123876
2016-04-06    0.220823
2016-04-07    0.130683
Name: last_seen, dtype: float64


We can see that with this column, the least number of ads last seen by the crawler were the first few dates the crawler ran. This column appears to indicate when a sale was completed. This makes sense, as the longer the crawler ran and the more ads it collected as time went on the greater the number of ads being removed would be.

The ad_created column contains the dates every ad was put up onto the website, and will likely contain many unique dates. Rather than viewing all the unique values down to the day, let's view them down to the month.

In [19]:
print(autos['ad_created'].str[:7].value_counts(normalize=True, dropna=False).sort_index(ascending=True))

2015-06    0.000020
2015-08    0.000020
2015-09    0.000020
2015-11    0.000020
2015-12    0.000040
2016-01    0.000241
2016-02    0.001265
2016-03    0.837490
2016-04    0.160884
Name: ad_created, dtype: float64


We can see that there still appears to be a few ads that are nearly a year old. There are very few of them, however, and are likely very bad deals or fake ads. Aside from these, 99% of all the ads were created the same months that our crawler ran. With less than 1% of ads still up from February, this indicates that most of the vehicles put up are either sold or removed within a month of being posted.

Finally, let's take a look at the registration_year column. Since this column is represented as a numeric dtype, we can simply use the Series.describe() method to view the range of our ads year of registration.

In [20]:
autos['registration_year'].describe()

count    49800.000000
mean      2005.074398
std        105.920807
min       1000.000000
25%       1999.000000
50%       2003.000000
75%       2008.000000
max       9999.000000
Name: registration_year, dtype: float64

This doesn't seem right. The minimum and maximum value in this column are 1000 and 9999. Both of these are impossible years for a vehicle to be registered. It's only possible that there could be ads with cars as old as the 1900s. Lets see how many listings are older than that.

In [21]:
autos.loc[autos['registration_year'] <= 1900, 'registration_year']

10556    1800
22316    1000
24511    1111
32585    1800
35238    1500
49283    1001
Name: registration_year, dtype: int64

There are only six rows with a value older than 1900, and they are all years before cars were invented. These rows can be removed to improve the accuracy of our data.

## Data Cleaning: Registration Year

Let's remove all rows with values under 1900 and over 2016, as anything outside of these values are impossible.

In [22]:
autos = autos[autos['registration_year'].between(1900,2016)]

print(autos['registration_year'].value_counts(normalize=True, dropna=False).head(20))

2000    0.070079
2005    0.063012
1999    0.062657
2004    0.057158
2003    0.056991
2006    0.056573
2001    0.056468
2002    0.052935
1998    0.051200
2007    0.048022
2008    0.046412
2009    0.043736
1997    0.042336
2011    0.033994
2010    0.033158
1996    0.030147
1995    0.027388
2016    0.027346
2012    0.027304
2013    0.016412
Name: registration_year, dtype: float64


We can see that newer cars 10-16 years old are the most commonly listed.

## Analysis: Common Brands

Let's find the average price and mileage of the top 10 most common brands in our dataset. We can do this with a simple aggregation loop. First, we'll find out what the most common brands are.

In [23]:
autos['brand'].value_counts().head(10)

volkswagen       10183
bmw               5269
opel              5194
mercedes_benz     4549
audi              4127
ford              3344
renault           2273
peugeot           1418
fiat              1242
seat               873
Name: brand, dtype: int64

### Mean Price

In [24]:
# This will be the dictionary that will hold the brand name and mean price
mean_price = {}

# This will be the list of our top 10 brands to loop through
brands = autos['brand'].value_counts().head(10).index

for b in brands: 
    brand = autos.loc[autos['brand'] == b, 'price']
    mean = brand.mean()
    mean_price[b] = int(mean)
    
# We'll sort the dictionary values to make them easier to read    
sorted(mean_price.items(), key=lambda x: x[1], reverse=True)

[('audi', 8788),
 ('mercedes_benz', 8054),
 ('bmw', 7870),
 ('volkswagen', 5219),
 ('seat', 4296),
 ('ford', 3526),
 ('peugeot', 3039),
 ('opel', 2876),
 ('fiat', 2711),
 ('renault', 2355)]

As we can see here, the three most costly brands are Audi, Mercedes Benz, and BMW. The two middle placed brands are Volkswagen and Seat. The cheapest brands are Ford, Peugeot, Opel, Fiat, and Renault.

The most popular brand, Volkswagen, has a mid tier cost. People seem to prefer vehicles that are in the middle of the price spectrum, and aren't super expensive or super cheap.

### Mean Mileage


In [25]:
# This will be the dictionary that will hold the brand name and mean mileage
mean_mileage = {}

# This will be the list of our top 10 brands to loop through
brands = autos['brand'].value_counts().head(10).index

for b in brands: 
    brand = autos.loc[autos['brand'] == b, 'odometer_km']
    mean = brand.mean()
    mean_mileage[b] = int(mean)
     
sorted(mean_mileage.items(), key=lambda x: x[1], reverse=True)

[('bmw', 132679),
 ('mercedes_benz', 131343),
 ('audi', 129806),
 ('opel', 129223),
 ('volkswagen', 128746),
 ('renault', 128227),
 ('peugeot', 127136),
 ('ford', 124197),
 ('seat', 121563),
 ('fiat', 116553)]

With both of our dictionaries seperated like this, it makes them a little difficult to compare. We can improve this by making them into a new DataFrame.

In [26]:
# We'll first make each dictionary a Series by using pandas series constructor
price_series = pd.Series(mean_price)
mileage_series = pd.Series(mean_mileage)

# Then we'll construct price_series into a DataFrame and add the mileage_series on as a column
common_brands = pd.DataFrame(price_series, columns=['mean_price'])
common_brands['mean_mileage'] = mileage_series

common_brands.sort_values('mean_mileage', ascending=False)

,mean_price,mean_mileage
bmw,7870,132679
mercedes_benz,8054,131343
audi,8788,129806
opel,2876,129223
volkswagen,5219,128746
renault,2355,128227
peugeot,3039,127136
ford,3526,124197
seat,4296,121563
fiat,2711,116553


With our new dataframe, we can easily see the relationship between the mean_price and mean_mileage for our top 10 brands. It appears that the more expensive brands tend to have more mileage, while the cheaper brands have less mileage.

## Analysis: Brand/Model Combination

We'll find the most popular models for the top 10 most popular brands as well. This can be easily done by aggregating through the popular brands with the `df.groupby()` method and finding the number of results in each group.

In [27]:
# Create a dictionary that will contain our results
brand_model = {}

for b in brands:
    # We'll want to create a new df for each brand
    new = autos[autos['brand'] == b]
    # Then find the most popular brand/model combination for that current aggregated brand
    new_series = new.groupby(['brand','model']).size().sort_values(ascending=False).head(1)
    # And finally, add the results to our dictionary
    brand_model[b] = new_series.loc[b].index[0]

brand_model

{'volkswagen': 'golf',
 'bmw': '3er',
 'opel': 'corsa',
 'mercedes_benz': 'c_klasse',
 'audi': 'a4',
 'ford': 'focus',
 'renault': 'twingo',
 'peugeot': '2_reihe',
 'fiat': 'punto',
 'seat': 'ibiza'}

These are the most popular brand/model combinations for our most popular brands. We can see that for our most popular brand, Volkswagen, the Volkswagen Golf is the most popular listed model.

## Analysis: Mileage Average Price

Does the mileage affect the average price? We can find this out by performing another simple aggregation. As we previously seen, there are only 13 unique values for the odometer_km column. This will make this process as simple as spliting each value into a group and finding the average price for each group.

In [28]:
# We'll create a new list with each unique mileage value
mileage_groups = autos['odometer_km'].value_counts().index

mileage_avg_price = {}

for m in mileage_groups:
    # Create a new df for each group using bool indexing
    new = autos[autos['odometer_km'] == m]
    # Find the mean for each unique group
    avg = new['price'].mean()
    # Apply our results to a dictionary
    mileage_avg_price[m] = int(avg)

# We'll convert the dictionary into a dataframe to increase readability
mileage_price = pd.DataFrame.from_dict(mileage_avg_price, orient='index', columns=["price"])
mileage_price.index.name = 'km'

# And sort the df by price
mileage_price.sort_values('price')

,price
km,
150000,3614
5000,4683
125000,5930
100000,7545
90000,8114
80000,9442
70000,10583
60000,11286
50000,12855


Our results seem to pretty definitely tell us that the more mileage a car has, the cheaper the car will be. This makes sense, as the more worn a car is, the less it is worth. This is very close to a linear progression, aside from the 5000 mileage outlier.

## Analysis: Damage/Price Comparison

For our last analysis, let's find out how much cheaper car brands are when they have unrepaired damage. We'll again use our top 10 most common car brands for this analysis.

In [29]:
# We'll need to create a dictionary to hold our values
damaged_comp = {}

# We already have a list variable with our top 10 brands, so we'll iterate on that
for b in brands:
    # Create two new dataframe to hold their respective information
    undamaged = autos[(autos['brand'] == b) & (autos['unrepaired_damage'] == 'no')]
    damaged = autos[(autos['brand'] == b) & (autos['unrepaired_damage'] == 'yes')]
    
    # Find the average price for each of our new dataframes
    undamaged_price = undamaged['price'].mean()
    damaged_price = damaged['price'].mean()
    
    # Each key in our dictionary will hold both values
    damaged_comp[b] = [int(undamaged_price), int(damaged_price)]
    
# And to make our data more readable, we'll convert it into a dataframe    
comp_df = pd.DataFrame.from_dict(damaged_comp, orient='index', columns=['undamaged', 'damaged'])

comp_df

,undamaged,damaged
volkswagen,6358,2037
bmw,9079,3276
opel,3607,1298
mercedes_benz,9157,3865
audi,10385,3181
ford,4517,1312
renault,2990,1073
peugeot,3636,1309
fiat,3424,1068
seat,5118,1610


From the information above, we can confidentally answer that cars with damage are usually around 3x as cheap as their undamaged counterparts.